In [43]:
import pandas as pd
import numpy as np

# Biblioteca para Normalização dos dados
from sklearn.preprocessing import StandardScaler

# Biblioteca para execução da Regressão
import statsmodels.api as sm

# Biblioteca para as devidas Visualizações
import seaborn as sns
import matplotlib.pyplot as plt

# TRatamento de multicolinearidade (camada complementar)
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

In [15]:
df = pd.read_excel("TabelaFatoMirante.xlsx", sheet_name="tabelaFato")

In [29]:
# Organização de dados
df = pd.read_excel("TabelaFatoMirante.xlsx", sheet_name="tabelaFato")

df = df[
    [
        "Leite Cru (R$/L)",
        "IPCA Alim.",
        "Dólar (R$)",
        "Milho (Saca)",
        "Diesel (R$/L)",
        "Rel. Troca (L/Saca)",
        "Codificação Clima"
    ]
]

df.columns = [
    "LeiteCru",
    "IPCA_Alim",
    "Dolar",
    "Milho",
    "Diesel",
    "Rel_Troca",
    "Clima"
]

In [32]:
# Normalização dos dados históricos
scaler = StandardScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [33]:
# Primeiros testes de correlação simples
print("=== Correlação Simples com Y (Leite Cru) ===")
for col in df.columns[1:]:
    corr = df["LeiteCru"].corr(df[col])
    print(f"Correlação LeiteCru x {col}: {corr:.4f}")

=== Correlação Simples com Y (Leite Cru) ===
Correlação LeiteCru x IPCA_Alim: -0.4391
Correlação LeiteCru x Dolar: 0.7103
Correlação LeiteCru x Milho: 0.3327
Correlação LeiteCru x Diesel: 0.6884
Correlação LeiteCru x Rel_Troca: -0.4601
Correlação LeiteCru x Clima: 0.5288


In [34]:
# Primeiros testes de correlação multipla
X = df_norm.drop(columns=["LeiteCru"])
y = df_norm["LeiteCru"]

X_const = sm.add_constant(X)
modelo_corr = sm.OLS(y, X_const).fit()

print("\n=== R² Múltiplo ===")
print(modelo_corr.rsquared)


=== R² Múltiplo ===
0.992270794012843


In [35]:
modelo = sm.OLS(y, X_const).fit()
print("\n=== Regressão Múltipla ===")
print(modelo.summary())


=== Regressão Múltipla ===
                            OLS Regression Results                            
Dep. Variable:               LeiteCru   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     385.1
Date:                Thu, 19 Feb 2026   Prob (F-statistic):           5.34e-18
Time:                        09:42:39   Log-Likelihood:                 25.311
No. Observations:                  25   AIC:                            -36.62
Df Residuals:                      18   BIC:                            -28.09
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.626e-15   

In [36]:
intercepto = modelo.params["const"]
coeficientes = modelo.params.drop("const")

print("\nIntercepto:", intercepto)
print("\nCoeficientes:")
print(coeficientes)


Intercepto: -1.6257011037941081e-15

Coeficientes:
IPCA_Alim    0.024727
Dolar        0.072879
Milho        1.174033
Diesel      -0.007122
Rel_Troca   -1.276439
Clima       -0.040782
dtype: float64


In [39]:
def simular_preco(ipca, dolar, milho, diesel, rel_troca, clima):
    return (
        intercepto
        + coeficientes["IPCA_Alim"] * ipca
        + coeficientes["Dolar"] * dolar
        + coeficientes["Milho"] * milho
        + coeficientes["Diesel"] * diesel
        + coeficientes["Rel_Troca"] * rel_troca
        + coeficientes["Clima"] * clima
    )

In [40]:
preco_simulado = simular_preco(
    ipca=0.25,
    dolar=5.10,
    milho=65,
    diesel=6.20,
    rel_troca=4.5,
    clima=2
)

print("\nPreço simulado do Leite Cru:", preco_simulado)


Preço simulado do Leite Cru: 70.82033832577524


In [47]:
# Tratando multicolinearidade
df = pd.read_excel("TabelaFatoMirante.xlsx", sheet_name="tabelaFato")

df = df[
    [
        "Leite Cru (R$/L)",
        "IPCA Alim.",
        "Dólar (R$)",
        "Milho (Saca)",
        "Diesel (R$/L)",
        "Rel. Troca (L/Saca)",
        "Codificação Clima"
    ]
]

df.columns = [
    "LeiteCru",
    "IPCA",
    "Dolar",
    "Milho",
    "Diesel",
    "RelTroca",
    "Clima"
]

# Normalizando
scaler = StandardScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Definindo Variável discreta Y e contínuas X
X = df_norm.drop(columns=["LeiteCru"])
y = df_norm["LeiteCru"]

# Testando multicolinearidade (VIF)
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["Variável"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)

   Variável        VIF
0      IPCA   2.302441
1     Dolar  18.235260
2     Milho  12.515812
3    Diesel  10.219413
4  RelTroca   4.859771
5     Clima   3.179025


In [50]:
# Regressão LASSO
lasso = Lasso(alpha=0.1)
lasso.fit(X, y)

coef_lasso = pd.Series(lasso.coef_, index=X.columns)
print("\nCoeficientes LASSO:")
print(coef_lasso)

# Regressão RIDGE
ridge = Ridge(alpha=1.0)
ridge.fit(X, y)

coef_ridge = pd.Series(ridge.coef_, index=X.columns)
print("\nCoeficientes Ridge:")
print(coef_ridge)



Coeficientes LASSO:
IPCA       -0.000000
Dolar       0.418104
Milho       0.170966
Diesel      0.176408
RelTroca   -0.586064
Clima       0.000000
dtype: float64

Coeficientes Ridge:
IPCA        0.006266
Dolar       0.210443
Milho       0.670237
Diesel      0.142385
RelTroca   -0.950670
Clima       0.012515
dtype: float64


In [51]:
elastic = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic.fit(X, y)

coef_elastic = pd.Series(elastic.coef_, index=X.columns)
print("\nCoeficientes Elastic Net:")
print(coef_elastic)


Coeficientes Elastic Net:
IPCA       -0.000000
Dolar       0.304967
Milho       0.347891
Diesel      0.202095
RelTroca   -0.708008
Clima       0.010145
dtype: float64


In [52]:
print("\nR² LASSO:", r2_score(y, lasso.predict(X)))
print("R² Ridge:", r2_score(y, ridge.predict(X)))
print("R² Elastic:", r2_score(y, elastic.predict(X)))


R² LASSO: 0.8800558392612715
R² Ridge: 0.968407866427114
R² Elastic: 0.9190397642429312


In [55]:
def simular_preco(ipca, dolar, milho, diesel, reltroca, clima):
    entrada = scaler.transform([[np.nan, ipca, dolar, milho, diesel, reltroca, clima]])[0][1:]
    return elastic.intercept_ + np.dot(elastic.coef_, entrada)

In [58]:
# Rodando LASSO com validação cruzada
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(cv=5, random_state=42)
lasso_cv.fit(X, y)

print("Melhor alpha LASSO:", lasso_cv.alpha_)
print("Coeficientes LASSO:")
print(pd.Series(lasso_cv.coef_, index=X.columns))

# Rodando Elastic com validação cruzada
from sklearn.linear_model import ElasticNetCV

elastic_cv = ElasticNetCV(cv=5, random_state=42, l1_ratio=[.1, .3, .5, .7, .9])
elastic_cv.fit(X, y)

print("Melhor alpha Elastic:", elastic_cv.alpha_)
print("Melhor l1_ratio:", elastic_cv.l1_ratio_)
print("Coeficientes Elastic:")
print(pd.Series(elastic_cv.coef_, index=X.columns))

Melhor alpha LASSO: 0.0007102694890358267
Coeficientes LASSO:
IPCA        0.020955
Dolar       0.064527
Milho       1.166975
Diesel      0.000000
RelTroca   -1.269915
Clima      -0.035311
dtype: float64
Melhor alpha Elastic: 0.0007891883211509187
Melhor l1_ratio: 0.9
Coeficientes Elastic:
IPCA        0.021026
Dolar       0.065563
Milho       1.165347
Diesel      0.000000
RelTroca   -1.268954
Clima      -0.035230
dtype: float64


In [59]:
from sklearn.model_selection import cross_val_score

scores_lasso = cross_val_score(lasso_cv, X, y, cv=5, scoring="r2")
scores_elastic = cross_val_score(elastic_cv, X, y, cv=5, scoring="r2")

print("R² LASSO (CV):", scores_lasso.mean())
print("R² Elastic (CV):", scores_elastic.mean())

R² LASSO (CV): 0.5105001387201782
R² Elastic (CV): 0.4671756085283219


In [62]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

# Carregar base
df = pd.read_excel("TabelaFatoMirante.xlsx", sheet_name="tabelaFato")

df = df[
    [
        "Leite Cru (R$/L)",
        "IPCA Alim.",
        "Dólar (R$)",
        "Milho (Saca)",
        "Diesel (R$/L)",
        "Rel. Troca (L/Saca)",
        "Codificação Clima"
    ]
]

df.columns = [
    "LeiteCru",
    "IPCA",
    "Dolar",
    "Milho",
    "Diesel",
    "RelTroca",
    "Clima"
]

scaler = StandardScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

X = df_norm.drop(columns=["LeiteCru"])
y = df_norm["LeiteCru"]


In [63]:
alphas = np.logspace(-3, 0, 50)  # de 0.001 a 1
l1_ratios = [0.1, 0.3, 0.5, 0.7]  # mais suaves, sem ir para 0.9/1.0

elastic_cv = ElasticNetCV(
    alphas=alphas,
    l1_ratio=l1_ratios,
    cv=5,
    random_state=42
)
elastic_cv.fit(X, y)

print("Melhor alpha:", elastic_cv.alpha_)
print("Melhor l1_ratio:", elastic_cv.l1_ratio_)
coef_elastic = pd.Series(elastic_cv.coef_, index=X.columns)
print("\nCoeficientes Elastic Net:")
print(coef_elastic)

Melhor alpha: 0.001
Melhor l1_ratio: 0.7

Coeficientes Elastic Net:
IPCA        0.021280
Dolar       0.068585
Milho       1.160735
Diesel      0.000000
RelTroca   -1.266275
Clima      -0.035066
dtype: float64


In [64]:
scores_elastic = cross_val_score(elastic_cv, X, y, cv=5, scoring="r2")
print("\nR² Elastic (CV):", scores_elastic.mean())


R² Elastic (CV): 0.5017821888539427


In [65]:
intercepto = elastic_cv.intercept_
coef = pd.Series(elastic_cv.coef_, index=X.columns)

print("\nIntercepto:", intercepto)
print("\nCoeficientes finais:")
print(coef)


Intercepto: -3.1421041768475445e-15

Coeficientes finais:
IPCA        0.021280
Dolar       0.068585
Milho       1.160735
Diesel      0.000000
RelTroca   -1.266275
Clima      -0.035066
dtype: float64


In [81]:
def simular_preco(ipca, dolar, milho, diesel, reltroca, clima):
    linha = pd.DataFrame(
        [[np.nan, ipca, dolar, milho, diesel, reltroca, clima]],
        columns=df.columns
    )
    
    linha_norm = pd.DataFrame(
        scaler.transform(linha),
        columns=df.columns
    )
    
    x_sim = linha_norm.drop(columns=["LeiteCru"]).values[0]
    
    y_norm = intercepto + np.dot(coef.values, x_sim)
    
    y_real = y_norm * scaler.scale_[0] + scaler.mean_[0]
    
    return round(y_real, 2)

In [82]:
preco = simular_preco(0.25, 5.10, 65, 6.20, 4.5, 2)
print(f"Preço simulado: R$ {preco:.2f}")

Preço simulado: R$ 4.68
